# Сбор данных

In [1]:
#processing
import pymysql.cursors 
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from pandas import ExcelWriter
from datetime import datetime
#import scikitplot as skplt
#from xgboost.sklearn import XGBClassifier
import scipy
from sklearn.preprocessing import OneHotEncoder
#import IDF
enc = OneHotEncoder()
scaler = StandardScaler()
label = LabelEncoder()
#NLP
#import nltk
#import re
#import pymorphy2
#from nltk.corpus import stopwords
#warnings
import warnings
warnings.filterwarnings('ignore')
#visualisation
#import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set(style="white", color_codes=True)
#%matplotlib inline
#vectorization
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#model 
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.metrics import roc_auc_score, roc_curve, log_loss, f1_score, confusion_matrix, precision_score, recall_score, classification_report, accuracy_score
#classificators
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [2]:
#Подключиться к базе данных
def con():
    conn = pymysql.connect(host='192.168.64.1', port=3306, user='i.serov', password='X3*1Uy(F', db='mysql')
    return conn

In [3]:
%%time
df_npl = pd.read_sql('''
select
    c.id as credit_id, c.date_requested,
                  ifnull(IF(c.status IN ('active','completed','expired'),
                  (SELECT
                    IF((COUNT(credit_payment.id) > 0 OR CURDATE() < LEAST(c.due_date,DATE(c.date_received) + INTERVAL 31 DAY) + INTERVAL 16 DAY),0,1)
                  FROM br_release_moneyman.credit_payment
                  WHERE ((credit_payment.credit_id = c.id) AND (credit_payment.payment_date <(LEAST(c.due_date,DATE(c.date_received) + INTERVAL 31 DAY) + INTERVAL 16 DAY)))),NULL),0)
                  AS npl15,
    pd.insurance_number
from br_release_moneyman.credit c
	left join br_release_moneyman.borrower b on c.borrower_id = b.id
	left join br_release_moneyman.personal_data pd on b.personal_data_id = pd.id
where c.status in ('ACTIVE', 'COMPLETED', 'EXPIRED', 'SOLD')
ORDER BY insurance_number
''', con=con())

Wall time: 2.5 s


In [4]:
%%time
inf = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 0)
inf.columns = 'DadosBasicos_'+ inf.columns
inf = inf.rename(columns={'DadosBasicos_Documento': 'Documento'})
ralat = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 1) #povt
ralat.columns = 'ralat_'+ ralat.columns
ralat2 = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 2) #povt
ralat2.columns = 'ralat2_'+ ralat2.columns
phones = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 4) #povt
phones.columns = 'phones_'+ phones.columns
emails = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 5) #povt
emails.columns = 'emails_'+ emails.columns
socdem = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 6)
socdem.columns = 'socdem_'+ socdem.columns
socdem = socdem.rename(columns={'socdem_Documento': 'Documento'})
soc_progr = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 7)
soc_progr.columns = 'soc_progr_'+ soc_progr.columns
soc_progr = soc_progr.rename(columns={'soc_progr_Documento': 'Documento'})
profess = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 8)
profess = profess.Documento
profess_big = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 9)
profess_big.columns = 'DadosProfissionais_ServidoresPu_'+ profess_big.columns
profess_big = profess_big.rename(columns={'DadosProfissionais_ServidoresPu_Documento': 'Documento'})
cobra = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 12)
cobra.columns = 'cobra_'+ cobra.columns
cobra = cobra.rename(columns={'cobra_Documento': 'Documento'})
finance = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 14)
finance.columns = 'finance_'+ finance.columns
finance = finance.rename(columns={'finance_Documento': 'Documento'})
present_online = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 17)
present_online.columns = 'present_online_'+ present_online.columns
present_online = present_online.rename(columns={'present_online_Documento': 'Documento'})
vendors = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 18) #povt
vendors.columns = 'vendors_'+ vendors.columns
insurance = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 19)
insurance.columns = 'insurance_'+ insurance.columns
insurance = insurance.rename(columns={'insurance_Documento': 'Documento'})
company = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 20)
company.columns = 'company_'+ company.columns
ABRT = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 21)  #povt
ABRT.columns = 'ABRT_'+ ABRT.columns
med_insur = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 22) #povt
med_insur.columns = 'med_insur_'+ med_insur.columns
inter = pd.read_excel('MoneyMan_BDC_Completo_12Dec17.xlsx', 23)
inter.columns = 'inter_'+ inter.columns
inter = inter.rename(columns={'inter_Documento': 'Documento'})

Wall time: 3min 18s


In [5]:
#company
company['company_Documento Empresa'] = company['company_Documento Empresa'].map(lambda x: 0 if x=='NO INFORMATION FOUND' else 1)
company = company[['company_Documento', 'company_Documento Empresa']]
company = company.rename(columns={'company_Documento': 'Documento'})

In [6]:
#relations
rlt1 = ralat.pivot_table(index = ['ralat_Documento'], columns = 'ralat_Tipo do Relacionamento', aggfunc='size').reset_index()
rlt1['ralat_Ultima Atualizacao'] = ralat.groupby('ralat_Documento').agg({'ralat_Ultima Atualizacao': 'max'}).reset_index()['ralat_Ultima Atualizacao']
relat = rlt1
relat = relat.rename(columns={'ralat_Documento': 'Documento'})
rlt2 = ralat2.pivot_table(index = ['ralat2_Documento'], columns = 'ralat2_Tipo do Relacionamento', aggfunc='size').reset_index()
rlt2['ralat2_Ultima Atualizacao'] = ralat2.groupby('ralat2_Documento').agg({'ralat2_Ultima Atualizacao': 'max'}).reset_index()['ralat2_Ultima Atualizacao']
relat2 = rlt2
relat2 = relat2.rename(columns={'ralat2_Documento': 'Documento'})

In [7]:
#phones
cnt = phones.groupby('phones_Documento').agg({'phones_Tipo': 'count'}).reset_index()
sm = phones.groupby('phones_Documento').agg({'phones_Passagens': 'sum'}).reset_index()
sm['phones_Ultima Atualizacao'] = phones.groupby('phones_Documento').agg({'phones_Ultima Atualizacao': 'max'}).reset_index()['phones_Ultima Atualizacao']
sm['phones_Primeira Passagem'] = phones.groupby('phones_Documento').agg({'phones_Primeira Passagem': 'max'}).reset_index()['phones_Primeira Passagem']
sm['phones_Ultima Passagem'] = phones.groupby('phones_Documento').agg({'phones_Ultima Passagem': 'max'}).reset_index()['phones_Ultima Passagem']
phones['phones_Pessoas no Telefone'] = phones['phones_Pessoas no Telefone'].replace('10+', 10).map(lambda x: int(x))
sm1 = phones.groupby('phones_Documento').agg({'phones_Pessoas no Telefone': 'sum'}).reset_index()
phones = pd.merge(cnt, pd.merge(sm1, sm, how = 'outer', on = 'phones_Documento'), how = 'outer', on = 'phones_Documento')
phones = phones.rename(columns={'phones_Tipo': 'phones_count'})
phones = phones.rename(columns={'phones_Documento': 'Documento'})

In [8]:
#emails
cnt = emails.groupby('emails_Documento').agg({'emails_Tipo': 'count'}).reset_index()
sm = emails.groupby('emails_Documento').agg({'emails_Passagens': 'sum'}).reset_index()
emails['emails_Pessoas no Email'] = emails['emails_Pessoas no Email'].replace('10+', 10).map(lambda x: int(x))
sm1 = emails.groupby('emails_Documento').agg({'emails_Pessoas no Email': 'sum'}).reset_index()
sm1['emails_Ultima Atualizacao'] = emails.groupby('emails_Documento').agg({'emails_Ultima Atualizacao': 'max'}).reset_index()['emails_Ultima Atualizacao']
sm1['emails_Primeira Passagem'] = emails.groupby('emails_Documento').agg({'emails_Primeira Passagem': 'max'}).reset_index()['emails_Primeira Passagem']
sm1['emails_Ultima Passagem'] = emails.groupby('emails_Documento').agg({'emails_Ultima Passagem': 'max'}).reset_index()['emails_Ultima Passagem']
emails = pd.merge(cnt, pd.merge(sm1, sm, how = 'outer', on = 'emails_Documento'), how = 'outer', on = 'emails_Documento')
emails = emails.rename(columns={'emails_Tipo': 'emails_count'})
emails = emails.rename(columns={'emails_Documento': 'Documento'})

In [9]:
#ABRT
ABRT['ABRT_DateLastUpdate'].fillna('01/01/2001', inplace = True)
ABRT['ABRT_DateLastUpdate'] = ABRT['ABRT_DateLastUpdate'].map(lambda x: datetime.strptime(x, '%d/%m/%Y'))
ABRT['ABRT_max_date'] = ABRT.groupby('ABRT_Documento')['ABRT_DateLastUpdate'].transform('max')
ABRT['ABRT_min_date'] = ABRT.groupby('ABRT_Documento')['ABRT_DateLastUpdate'].transform('min')
ABRT['ABRT_OfficialName'] = ABRT.groupby(['ABRT_Documento'])['ABRT_OfficialName'].transform('count')
ABRT = ABRT[['ABRT_Documento', 'ABRT_OfficialName', 'ABRT_max_date', 'ABRT_min_date']].drop_duplicates()
ABRT = ABRT.rename(columns={'ABRT_Documento': 'Documento'})

In [10]:
#med insurance
med_insur = pd.concat(
    [med_insur, pd.get_dummies(med_insur.med_insur_Status)], axis=1)
med_insur['med_insur_Nome'] = med_insur.groupby('med_insur_Documento')[
    'med_insur_Nome'].transform('count')
med_insur.med_insur_Operadora.fillna('noooo - noooo', inplace=True)
med_insur = med_insur[med_insur.med_insur_Operadora != 'noooo - noooo']
med_insur.med_insur_Operadora = med_insur.med_insur_Operadora.map(lambda x: x.split('-')[1])
med_insur.med_insur_Operadora = label.fit_transform(
    med_insur.med_insur_Operadora)
med_insur['med_insur_Plano de Saude'] = med_insur[
    'med_insur_Plano de Saude'].map(lambda x: x.split('-')[1])
med_insur['med_insur_Plano de Saude'] = med_insur.groupby(
    'med_insur_Plano de Saude')['med_insur_Plano de Saude'].transform('count')
med_insur['med_insur_Caracteristicas do Plano'] = med_insur.groupby(
    'med_insur_Caracteristicas do Plano')[
        'med_insur_Caracteristicas do Plano'].transform('count')
med_insur = med_insur[[
    'med_insur_Documento', 'med_insur_Nome', 'med_insur_Status',
    'med_insur_Data de Nascimento', 'med_insur_Operadora',
    'med_insur_Plano de Saude', 'med_insur_Caracteristicas do Plano'
]].sort_values('med_insur_Documento')
med_insur['max_med_insur_Operadora'] = med_insur.groupby(
    'med_insur_Documento')['med_insur_Operadora'].transform('max')
med_insur['min_med_insur_Operadora'] = med_insur.groupby(
    'med_insur_Documento')['med_insur_Operadora'].transform('min')
med_insur['max_med_insur_Plano_de_Saude'] = med_insur.groupby('med_insur_Documento')[
    'med_insur_Plano de Saude'].transform('max')
med_insur['min_med_insur_Plano_de_Saude'] = med_insur.groupby('med_insur_Documento')[
    'med_insur_Plano de Saude'].transform('min')
med_insur['max_med_insur_Caracteristicas'] = med_insur.groupby('med_insur_Documento')[
    'med_insur_Caracteristicas do Plano'].transform('max')
med_insur['min_med_insur_Caracteristicas'] = med_insur.groupby('med_insur_Documento')[
    'med_insur_Caracteristicas do Plano'].transform('min')
med_insur = med_insur[[
    'med_insur_Documento', 'med_insur_Nome', 'med_insur_Status',
    'med_insur_Data de Nascimento', 'max_med_insur_Operadora',
    'min_med_insur_Operadora', 'max_med_insur_Plano_de_Saude',
    'min_med_insur_Plano_de_Saude', 'max_med_insur_Caracteristicas',
    'min_med_insur_Caracteristicas'
]].drop_duplicates()
med_insur['med_insur_Data de Nascimento'] = med_insur['med_insur_Data de Nascimento'].map(
    lambda x: datetime.strptime(x, '%d/%m/%Y'))
med_insur['med_insur_Data de Nascimento'] = med_insur['med_insur_Data de Nascimento'].map(
    lambda x: 2017 - x.year)
med_insur = med_insur.rename(columns={'med_insur_Documento': 'Documento'})

In [11]:
vendors['vendors_Data do Primeiro Anuncio'] = vendors.groupby('vendors_Documento')['vendors_Data do Primeiro Anuncio'].transform('min')
vendors['vendors_Data do Ultimo Anuncio'] = vendors.groupby('vendors_Documento')['vendors_Data do Ultimo Anuncio'].transform('max')
vendors['vendors_Total de Anuncios'] = vendors.groupby('vendors_Documento')['vendors_Total de Anuncios'].transform('mean')
vendors['vendors_Maior Valor de Anuncio'] = vendors.groupby('vendors_Documento')['vendors_Maior Valor de Anuncio'].transform('max')
vendors['vendors_Menor Valor de Anuncio'] = vendors.groupby('vendors_Documento')['vendors_Menor Valor de Anuncio'].transform('min')
vendors['vendors_Valor Medio de Anuncio'] = vendors.groupby('vendors_Documento')['vendors_Valor Medio de Anuncio'].transform('mean')
vendors['vendors_Anuncios na Categoria Principal'] = vendors.groupby('vendors_Documento')['vendors_Anuncios na Categoria Principal'].transform('mean')
vendors['vendors_Valor do Ultimo Anuncio'] = vendors.groupby('vendors_Documento')['vendors_Valor do Ultimo Anuncio'].transform('max')
vendors.vendors_Categorias = vendors.vendors_Categorias + ';'
vendors.vendors_Categorias = vendors.groupby('vendors_Documento')['vendors_Categorias'].transform('sum')
vendors['vendors_Categoria Principal'] = vendors['vendors_Categoria Principal'] + ';'
vendors['vendors_Categoria Principal'] = vendors.groupby('vendors_Documento')['vendors_Categoria Principal'].transform('sum')
vendors['vendors_Anuncios Ativos'] = vendors.groupby('vendors_Documento')['vendors_Anuncios Ativos'].transform('max')
vendors['vendors_Anuncios nos Ultimos 30 Dias'] = vendors.groupby('vendors_Documento')['vendors_Anuncios nos Ultimos 30 Dias'].transform('max')
vendors['vendors_Anuncios nos Ultimos 90 Dias'] = vendors.groupby('vendors_Documento')['vendors_Anuncios nos Ultimos 90 Dias'].transform('max')
vendors['vendors_Anuncios nos Ultimos 180 Dias'] = vendors.groupby('vendors_Documento')['vendors_Anuncios nos Ultimos 180 Dias'].transform('max')
vendors['vendors_Anuncios nos Ultimos 365 Dias'] = vendors.groupby('vendors_Documento')['vendors_Anuncios nos Ultimos 365 Dias'].transform('max')
vendors['vendors_Anuncios na Categoria Principal'] = vendors.groupby('vendors_Documento')['vendors_Anuncios na Categoria Principal'].transform('max')
vendors['vendors_Anuncios no Portal Principal'] = vendors.groupby('vendors_Documento')['vendors_Anuncios no Portal Principal'].transform('max')
vendors = vendors.drop('vendors_Telefone', axis=1).drop_duplicates()
vendors = vendors[(vendors.vendors_Documento!=22631908805) &  (vendors.vendors_Documento!=13960271735)]
vendors = vendors.rename(columns={'vendors_Documento': 'Documento'})

In [12]:
df_jn = pd.merge(
    relat,
    pd.merge(
        inf,
        pd.merge(
            phones,
            pd.merge(
                emails,
                pd.merge(
                    socdem,
                    pd.merge(
                        soc_progr,
                        pd.merge(
                            profess_big,
                            pd.merge(
                                cobra,
                                pd.merge(
                                    present_online,
                                    pd.merge(
                                        vendors,
                                        pd.merge(
                                            insurance,
                                            pd.merge(
                                                company,
                                                pd.merge(
                                                    ABRT,
                                                    pd.merge(
                                                        med_insur,
                                                        pd.merge(
                                                            inter,
                                                            relat2,
                                                            how='outer',
                                                            on='Documento'
                                                        ),
                                                        how='outer',
                                                        on='Documento'),
                                                    how='outer',
                                                    on='Documento'),
                                                how='outer',
                                                on='Documento'),
                                            how='outer',
                                            on='Documento'),
                                        how='outer',
                                        on='Documento'),
                                    how='outer',
                                    on='Documento'),
                                how='outer',
                                on='Documento'),
                            how='outer',
                            on='Documento'),
                        how='outer',
                        on='Documento'),
                    how='outer',
                    on='Documento'),
                how='outer',
                on='Documento'),
            how='outer',
            on='Documento'),
        how='outer',
        on='Documento'),
    how='outer',
    on='Documento')

In [13]:
df = df_jn.copy()
#df_jn = df.copy()

In [14]:
docs_list = pd.read_excel('cpf_formatted.xlsx')

In [15]:
df = pd.merge(df, docs_list, how='left', on='Documento').drop(
    ['q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'Documento'], axis=1)
df = df.rename(columns={'Documento formated': 'Documento'})
df_npl = df_npl.rename(columns={'insurance_number': 'Documento'})
df = pd.merge(df_npl, df, how='left', on='Documento')

In [16]:
df = df.drop_duplicates()

In [17]:
to_drop = ['DadosBasicos_Nome', 'DadosBasicos_Nome da Mae', 'DadosProfissionais_ServidoresPu_Ocupacao',
       'DadosProfissionais_ServidoresPu_Entidade Governamental', 'DadosProfissionais_ServidoresPu_Ministerio',
          'vendors_Portais','vendors_Categoria Principal',
       'vendors_Categorias', 'vendors_Portal Principal',
           'insurance_Nome', 'insurance_PIS',
      'insurance_Motivo',
       'insurance_Procedimento', 'insurance_Parcelas', 'insurance_Situaçőes',
       'insurance_Datas de Disponibilidade', 'ABRT_max_date', 'ABRT_min_date']
df = df.drop(to_drop, axis=1)

In [18]:
dates_to_check_1 = [
    #inf
    'DadosBasicos_Data de Nascimento','DadosBasicos_Data de Criacao','DadosBasicos_Ultima Atualizacao',
    #relat12
    'ralat_Ultima Atualizacao','ralat2_Ultima Atualizacao',
    #phones
    'phones_Ultima Atualizacao', 'phones_Primeira Passagem', 'phones_Ultima Passagem',
    #emails
    'emails_Ultima Atualizacao', 'emails_Primeira Passagem','emails_Ultima Passagem', 
    #socdem
    'socdem_Data de Criacao', 'socdem_Ultima Atualizacao',
    #cobra
    'cobra_Data da Primeira Ocorrencia', 'cobra_Data da Ultima Ocorrencia',
    #pres onl
    'present_online_Primeira Passagem', 'present_online_Ultima Passagem',
    #vendors
    'vendors_Data do Primeiro Anuncio', 'vendors_Data do Ultimo Anuncio', 
]
dates_to_check_2 = [
    #profes big
   'DadosProfissionais_ServidoresPu_Data de Criacao', 'DadosProfissionais_ServidoresPu_Ultima Atualizacao'  
]

In [20]:
def date_prep_1(columns):
    for i in columns:
        df[i] = df[i].replace('0001-01-01', '01-01-2001')
        df[i] = df[i].replace('01-01-0001', '01-01-2001')        
        df[i] = df[i].fillna('01-01-2001')
        df[i] = df[i].map(lambda x: datetime.strptime(x, '%d-%m-%Y'))
        df[i] = df.date_requested - df[i]
        df[i] = df[i].map(lambda x: x.days)
        
def date_prep_2(columns):
    for i in columns:
        df[i] = df[i].replace('0001-01-01', '2001-01-01')
        df[i] = df[i].replace('01-01-0001', '2001-01-01')        
        df[i] = df[i].fillna('2001-01-01')
        df[i] = df[i].map(lambda x: datetime.strptime(x, '%Y-%m-%d'))
        df[i] = df.date_requested - df[i]
        df[i] = df[i].map(lambda x: x.days)

In [21]:
date_prep_1(dates_to_check_1)
date_prep_2(dates_to_check_2)

In [22]:
df.fillna(-1, inplace=True)

In [25]:
X = df[['credit_id', 'date_requested', 'Documento', 'BROTHER', 'HOUSEHOLD','NEIGHBOR','DadosBasicos_Signo',
       'DadosProfissionais_ServidoresPu_Ultima Atualizacao', 'emails_Primeira Passagem','phones_Pessoas no Telefone',
      'phones_Ultima Atualizacao','phones_Primeira Passagem','emails_Ultima Atualizacao','ralat_Ultima Atualizacao',
      'present_online_Passagens','cobra_Data da Primeira Ocorrencia', 'cobra_Data da Ultima Ocorrencia',
      'phones_Passagens','DadosBasicos_Ultima Atualizacao','phones_Ultima Passagem','emails_Ultima Passagem']]

In [27]:
writer = pd.ExcelWriter('Bigdata_vars.xlsx')
X.to_excel(writer,'Bigdata_vars', index=False)
writer.save()

In [ ]:
'''cat_vars = ['DadosBasicos_Status','DadosBasicos_Genero', 'DadosBasicos_Signo', 'DadosBasicos_Obito','socdem_Classe Social','socdem_Renda Estimada - BDC', 'socdem_Renda Media - IBGE',
       'socdem_Renda Media - MTE', 'socdem_Escolaridade Media - IBGE',
       'socdem_Escolaridade Media - MTE', 'DadosProfissionais_ServidoresPu_Faixa Salarial','DadosProfissionais_ServidoresPu_UF', 
           'finance_Renda Estimada',   'finance_Status da Restituicao', 'finance_Banco da Restituicao',
           'present_online_Comprador Online','med_insur_Status',
       'present_online_Vendedor Online','insurance_Situaçăo', 'insurance_Tempo de Serviço',
           'inter_Nivel de Utilizacao de Cartao de Credito', 'inter_Programas de Fidelidade']
for i in cat_vars:
    df[i] = label.fit_transform(df[i].astype(str))''';